In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
df1=pd.read_csv("C:\\Users\\HP\\Downloads\\hacktrain.csv")
df2=pd.read_csv("C:\\Users\\HP\\Downloads\\hacktest.csv")

In [3]:
df1.isnull().sum()
df1.fillna(df1.mean(numeric_only=True), inplace=True) #simple mean imputation [This part has a lot of scope for imporovement.]
#keep in mind that the data is inherently noisy and the test dataset is not.
df1.isnull().sum()

Unnamed: 0    0
ID            0
class         0
20150720_N    0
20150602_N    0
20150517_N    0
20150501_N    0
20150415_N    0
20150330_N    0
20150314_N    0
20150226_N    0
20150210_N    0
20150125_N    0
20150109_N    0
20141117_N    0
20141101_N    0
20141016_N    0
20140930_N    0
20140813_N    0
20140626_N    0
20140610_N    0
20140525_N    0
20140509_N    0
20140423_N    0
20140407_N    0
20140322_N    0
20140218_N    0
20140202_N    0
20140117_N    0
20140101_N    0
dtype: int64

In [4]:
df1.drop(columns=['ID'], inplace=True)

# Encode class column (if it's categorical)
label_encoder = LabelEncoder()
df1['class'] = label_encoder.fit_transform(df1['class'])

# Split into features and target
x = df1.drop(columns=['class'])
y = df1['class']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y)


In [6]:
preprocessor = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

In [7]:
X_train_processed = preprocessor.fit_transform(X_train)



In [8]:

# Transform test data using the same fitted preprocessor
X_test_processed = preprocessor.transform(X_test)

In [9]:
model = LogisticRegression(
    multi_class="multinomial",
    solver="lbfgs",
    max_iter=500
)
model.fit(X_train_processed, y_train)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=500, multi_class='multinomial')

In [10]:
y_pred = model.predict(X_test_processed)

In [11]:
print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder.classes_))),
    target_names=label_encoder.classes_
))


              precision    recall  f1-score   support

        farm       0.85      0.88      0.86       168
      forest       1.00      0.99      0.99      1232
       grass       0.83      0.64      0.72        39
  impervious       0.83      0.89      0.86       134
     orchard       1.00      0.50      0.67         6
       water       0.89      0.81      0.85        21

    accuracy                           0.96      1600
   macro avg       0.90      0.78      0.83      1600
weighted avg       0.96      0.96      0.96      1600



In [12]:
df2.shape

(2845, 29)

In [13]:
ID=df2['ID']
df2.drop(['ID'],axis=1,inplace=True)

In [14]:
y_test = model.predict(df2)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [15]:
y_test

array([4, 4, 4, ..., 2, 2, 2])

In [16]:
y_decoded = label_encoder.inverse_transform(y_test)
y_decoded

array(['orchard', 'orchard', 'orchard', ..., 'grass', 'grass', 'grass'],
      dtype=object)

In [17]:
result = pd.DataFrame({
    'ID': ID,
    'class': y_decoded
})

In [18]:
result

,ID,class
0,1,orchard
1,2,orchard
2,3,orchard
3,4,orchard
4,5,orchard
...,...,...
2840,2841,grass
2841,2842,grass
2842,2843,grass
2843,2844,grass


In [22]:
result.to_csv("summeranalytics.csv", index=False)

In [23]:
import os
os.getcwd()


'C:\\Users\\HP'

In [39]:
print(model.n_iter_)


[59]


In [40]:
from sklearn.metrics import accuracy_score
